In [12]:
import pandas as pd
import numpy as np
from collections import Counter
import csv
import nltk
import re
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import pymorphy2
import seaborn as sns
sns.set_style("darkgrid")
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Justin
[nltk_data]     Walker\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
class website:
    def __init__(self, CSV_POS='', CSV_NEG='', text_pos={}, text_neg={},
                 tf_idf_matrix_pos='', tf_idf_matrix_neg='', web_name='',
                 words_pos=[], words_neg=[]):
        self.csv = {
            "pos": CSV_POS,
            "neg": CSV_NEG
        }
        self.text = {
            "pos": text_pos,
            "neg": text_neg
        }
        self.tfidf_matrix = {
            "pos":  tf_idf_matrix_pos,
            "neg": tf_idf_matrix_neg
        }
        self.name = web_name
        self.words = {
            "pos": words_pos,
            "neg": words_neg
        }

In [14]:
def list_to_str(arr):
    s = ''
    for rec in arr:
        s += rec
    return df_preprocess_revs(s)

In [15]:
def csv_to_list(arr):
    arr_list = []
    for row in arr:
        arr_list.append(list_to_str(row))
    return arr_list

In [16]:
def key(x):
    try:
        return countt[x]
    except KeyError:
        return 0

In [17]:
def clasterization(w, st):
    print(f'\t{st} reviews from {w.name}: clasterization...')
    num_clusters = 10
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    kmeans.fit(w.tfidf_matrix[st])
    for cluster_id in range(num_clusters):
        cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
        print(f'\nКластер {cluster_id + 1}: (кол-во отзывов: {len(cluster_indices)}) ------------\n')
        ch = 0
        for idx in sorted(cluster_indices, key=lambda x: countt[w.words[st][x]], reverse=True):
            if ch == 20:
                break
            if (rev_list[idx] != ''):
                print(f"{rev_list[idx][:-12:]}: {countt[w.words[st][idx]]}")
            ch += 1
    print("--------")

In [18]:
def vectorization(w, st):
    print(f'\t{st} reviews from {w.name}: vectorization...')
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(w.text[st]['Text'])
    count_reviews = vectors.shape[0]
    count_tockens = vectors.shape[1]

    tf_idf = pd.DataFrame(vectors.todense()).iloc[:count_reviews]
    tf_idf.columns = vectorizer.get_feature_names_out()
    w.words[st] = tf_idf.columns    # сохранение для кластеризации
    tfidf_matrix = tf_idf.T
    tfidf_matrix.columns = ['response' + str(i) for i in range(1, count_reviews + 1)]
    tfidf_matrix['count'] = tfidf_matrix.sum(axis=1)
    for i in range(len(tfidf_matrix['count'])):
        countt[w.words[st][i]] = tfidf_matrix['count'][i]

    tfidf_matrix = tfidf_matrix.sort_values(by ='count', ascending=False)
    print(tfidf_matrix.drop(columns=['count']).head(count_tockens))
    return tfidf_matrix.T

In [19]:
def df_preprocess(text):
    reg = re.compile(r'[^а-яА-яa-zA-Z0-9 ]')
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text)
    text = re.sub(r'@[^\s]+', 'пользователь', text)
    text = reg.sub(' ', text)
    stopWords = set(stopwords.words('russian'))
    # Стемминг
    stemmer = SnowballStemmer("russian")
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopWords])
    return text

In [20]:
def df_preprocess_revs(text):
    reg = re.compile(r'[^а-яА-яa-zA-Z0-9 ]')
    text = reg.sub(' ', text)
    return text

In [21]:
statuses = ["neg", "pos"]

In [22]:
# websites init
rustore = website()
rustore.name = 'rustore.ru'
for st in statuses:
    rustore.csv[st] = fr'result\api\rustore.ru\csv\revData_rustore.ru_{st}.csv'
    rustore.text[st] = pd.read_csv(rustore.csv[st])
    rustore.text[st]['Text'] = rustore.text[st]['Text'].apply(df_preprocess)

In [23]:
sravni = website()
sravni.name = 'sravni.ru'
for st in statuses:
    sravni.csv[st] = fr'result\api\sravni.ru\csv\revData_sravni.ru_{st}.csv'
    sravni.text[st] = pd.read_csv(sravni.csv[st])
    sravni.text[st]['Text'] = sravni.text[st]['Text'].apply(df_preprocess)

In [ ]:
for st in statuses:
    countt = {}
    rustore.tfidf_matrix[st] = vectorization(rustore, st)
    with open(rustore.csv[st], 'r', encoding='utf-8') as file:
        q = csv.reader(file)
        rev_list = csv_to_list(q)
    clasterization(rustore, st)
    sravni.tfidf_matrix[st] = vectorization(sravni, st)
    with open(sravni.csv[st], 'r', encoding='utf-8') as file:
        q = csv.reader(file)
        rev_list = csv_to_list(q)
    clasterization(sravni, st)

	neg reviews from rustore.ru: vectorization...
          response1  response2  response3  response4  response5  response6  \
приложен        0.0   0.189789        0.0   0.000000   0.000000   0.000000   
работа          0.0   0.000000        0.0   0.272432   0.157677   0.000000   
банк            0.0   0.000000        0.0   0.216813   0.000000   0.000000   
открыва         0.0   0.000000        0.0   0.000000   0.000000   0.262239   
обновлен        0.0   0.000000        0.0   0.000000   0.000000   0.000000   
...             ...        ...        ...        ...        ...        ...   
дмитр           0.0   0.000000        0.0   0.000000   0.000000   0.000000   
пачк            0.0   0.000000        0.0   0.000000   0.000000   0.000000   
58              0.0   0.000000        0.0   0.000000   0.000000   0.000000   
странно         0.0   0.000000        0.0   0.000000   0.000000   0.000000   
выпаст          0.0   0.000000        0.0   0.000000   0.000000   0.000000   

          respon

C:\Users\Justin Walker\AppData\Local\Temp\ipykernel_14744\380196794.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  countt[w.words[st][i]] = tfidf_matrix['count'][i]


	neg reviews from rustore.ru: clasterization...

Кластер 1: (кол-во отзывов: 876) ------------

Самое плохое приложение среди банков: 70.18192461957446
Приложение перестало работать выбивать постоянно очень сильно лагает если удалось зайти: 26.06133769713229
Не открывается за границей  Уже два года  Все остальные   ВТБ к примеру  Сбер  или какой нибудь  Озон   открываются без проблем : 17.52176939670447
Говнопрмложение      Долбонбская реализация двухфакторуи  Сообщения идут по часу: 17.273500198170865
Сегодня обновил приложение  На покупку с вкус вил повышенный кешбек не начисляется  И приложение стало зависать или вообще не входит горит ж лтый экран и вс : 17.21444392612127
Можно премию дарвина вручить дизайнеру приложения  Как вообще во вс м этом награмождении бесконечных иконок и менюшек найти категории повышенного кэшбека на месяц  Каждый месяц  при заходе в приложение  пока ты одуплишь вбить в поиск повышенный кэшбек  все нервы потратишь  Ну и который год уже не добавлять вход по

C:\Users\Justin Walker\AppData\Local\Temp\ipykernel_14744\380196794.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  countt[w.words[st][i]] = tfidf_matrix['count'][i]


          response1  response2  response3  response4  response5  response6  \
банк       0.063191    0.05479   0.098311   0.032261   0.066216   0.023490   
карт       0.043310    0.00000   0.067381   0.088445   0.045383   0.000000   
эт         0.000000    0.00000   0.000000   0.069348   0.047446   0.050495   
тинькофф   0.103766    0.00000   0.000000   0.026488   0.000000   0.038573   
мо         0.100811    0.00000   0.000000   0.025733   0.052818   0.018738   
...             ...        ...        ...        ...        ...        ...   
service    0.000000    0.00000   0.000000   0.000000   0.000000   0.000000   
senior     0.000000    0.00000   0.000000   0.000000   0.000000   0.000000   
said       0.000000    0.00000   0.000000   0.000000   0.000000   0.000000   
refused    0.000000    0.00000   0.000000   0.000000   0.000000   0.000000   
except     0.000000    0.00000   0.000000   0.000000   0.000000   0.000000   

          response7  response8  response9  response10  ...  res

C:\Users\Justin Walker\AppData\Local\Temp\ipykernel_14744\380196794.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  countt[w.words[st][i]] = tfidf_matrix['count'][i]



Кластер 1: (кол-во отзывов: 1) ------------

Да: 0.3402567194567016

Кластер 2: (кол-во отзывов: 1) ------------


Кластер 3: (кол-во отзывов: 46) ------------

Отличное понятное приложени: 9.357951980843831
Почему пропал вход по отпечатку пальца   В настройках при включении пишет  что устройство поддерживает вход только по коду   а с другими приложениями проблем нет: 1.8868780089410297
Хорошее приложение: 1.7901740802046973
Ну  Тинькофф по праву носит звание лучшего мобильного банка   приложение действительно достойное и довольно удобное  Плюс  мне нравится  что есть удобные и  к месту  раздел с партн рскими кэшбэками  Тинькофф Журнал и разделом Город  где можно заказать доставку продуктов правда пока выбор невелик   Я бы очень хотел только иметь возможность настраивать отображение истории операций  чтоб иметь возможность какие то отсеивать например переводы туда сюда   но это не критично  Видно  что крутые ребята: 1.6143319763874757
В целом неплохо   Техподдержка радует оперативност

C:\Users\Justin Walker\AppData\Local\Temp\ipykernel_14744\380196794.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  countt[w.words[st][i]] = tfidf_matrix['count'][i]


          response1  response2  response3  response4  response5  response6  \
банк       0.031350   0.000000   0.086189   0.000000   0.059030   0.023628   
карт       0.087898   0.168354   0.000000   0.087317   0.000000   0.000000   
тинькофф   0.000000   0.000000   0.000000   0.000000   0.000000   0.065118   
очен       0.000000   0.000000   0.000000   0.047330   0.000000   0.000000   
лучш       0.000000   0.000000   0.000000   0.000000   0.116124   0.046480   
...             ...        ...        ...        ...        ...        ...   
special    0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
absence    0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
saved      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
resolve    0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
solid      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   

          response7  response8  response9  response10  ...  res